In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
df = pd.read_csv('housing.csv')

ENCODING THE ocean_proximity CATEGORICAL VARIABLE

In [3]:
def encode_proximity(x):
    location = x['ocean_proximity']
    if location == 'NEAR BAY':
        return 0
    elif location == '<1H OCEAN':
        return 1
    elif location == 'INLAND':
        return 2
    elif location == 'NEAR OCEAN':
        return 3
    elif location == 'ISLAND':
        return 4
    else:
        raise Exception
df['ocean_proximity'] = df.apply(encode_proximity, axis = 1)

# PREPROCESSING

FILLING OUT nan's IN total_bedrooms

In [4]:
df['total_bedrooms'] = df['total_bedrooms'].fillna(df['total_bedrooms'].mean())

APPLYING Z-SCALE NORMALIZATION TO THE median_house_value

In [5]:
MEAN = df['median_house_value'].mean()
STD = df['median_house_value'].std()

df['median_house_value'] = df.apply(lambda x: (x['median_house_value'] - MEAN) / STD, axis = 1)

APPLYING Z-SCALE NORMALIZATION TO total_rooms, total_bedrooms, population, households AND median_income
AVOIDING APPLYING NORMALIZATION TO CATEGORICAL VARIABLE ocean_proximity

In [6]:
columns = df.columns.to_list()[:-2]
for col in columns:
    MEAN = df[col].mean()
    STD = df[col].std()
    df[col] = df.apply(lambda x: (x[col] - MEAN) / STD, axis = 1)

SEPARATING THE DATASET INTO label(Y) AND input_features(X)

In [7]:
train = df.sample(frac = 0.7, random_state = 24075108)
combined = df.drop(train.index)
test = combined.sample(frac = 2 / 3, random_state = 24075108)
val = combined.drop(test.index)

In [8]:
train_Y = train['median_house_value'].values
train_X = train.loc[:, ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity']].values

test_Y = test['median_house_value'].values
test_X = test.loc[:, ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity']].values

val_Y = val['median_house_value'].values
val_X = val.loc[:, ['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity']].values

# STARTING TRAINING

In [9]:
THETA =[0.0 for x in range(train_X.shape[1] + 1)]

In [10]:
LEARNING_RATE = 0.01

def h(THETA, xi):
    partial = 0.0
    xi = np.insert(xi, 0, 1)
    for i in range(len(THETA)):
        partial += THETA[i] * xi[i]
    return partial

def cost(THETA, X, Y):
    res = 0.0
    m = Y.size
    for idx, _ in enumerate(X):
        h_of_theta = h(THETA, _)
        res += (h_of_theta - Y[idx]) ** 2
    return (1 / (2 * m)) * res

def r2(THETA, X, Y):
    m = Y.size
    RSS = 2 * m * cost(THETA, X, Y)
    TSS = sum([y ** 2 for y in Y])
    # As after normalization, mean of Y becomes 0
    return 1 - (RSS / TSS)

def r2_preds_true(Y_pred, Y_true):
    m = Y_pred.size
    RSS = np.sum(np.power(Y_pred - Y_true, 2))
    TSS = np.sum(np.power(Y_true.mean() - Y_true, 2))
    return 1 - (RSS / TSS)

In [11]:
def update_theta_j(THETA, X, Y, j):
    m = Y.size
    del_J_theta_j = 0.0
    for i in range(m):
        xi = np.insert(X[i], 0, 1)
        del_J_theta_j = del_J_theta_j + xi[j] * (h(THETA, X[i]) - Y[i])
    del_J_theta_j = del_J_theta_j * (LEARNING_RATE / m)
    return del_J_theta_j
    
def update_theta(THETA, X, Y):
    THETA_OLD = THETA.copy()
    
    n = len(THETA)
    for i in range(n):
        THETA[i] -= update_theta_j(THETA_OLD, X, Y, i)
    return THETA

In [20]:
best_THETA = THETA.copy()
PATIENCE_LIMIT = 10
patience_counter = 0
best_cost_VAL = np.float64('inf')

for i in range(1000):
    start_epoch = time.time()
    THETA = update_theta(THETA, train_X, train_Y)
    end_epoch = time.time()
    cost_VAL = cost(THETA, val_X, val_Y)
    cost_TRAIN = cost(THETA, train_X, train_Y)

    if (cost_VAL < best_cost_VAL):
        patience_counter = 0
        best_THETA = THETA.copy()
        best_cost_VAL = cost_VAL
    else:
        patience_counter += 1
               
    if (patience_counter == PATIENCE_LIMIT):
        break

KeyboardInterrupt: 